# Preparing Data

In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
base_dir= "/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Datasets-v2/"
laptops = "laptops_cleaned.json"
#meta_ranges = "meta_Ranges_Ovens_Cooktops.json"
#meta_laundry = "meta_Laundry_Appliancess.json"
#apl = "/content/drive/My Drive/Cal State LA/Machine Learning/Datasets/Appliances.json"

In [3]:
df_laptops=pd.read_json(base_dir+laptops)
#df_apl = pd.read_json(apl, lines=True)
#df_laundry_joined = pd.merge(df_meta_laundry, df_apl, how='inner', on="asin")
#df_laundry_joined = df_laundry_joined.drop(columns=['image_y', 'date', 'feature','unixReviewTime', 'summary','reviewerName','style','reviewerID','reviewTime','verified','C1', 'title', 'also_buy', 'tech2', 'category_len','brand','rank','also_view','similar_item','details','main_cat','tech1','image_x'])

In [4]:
df_laptops.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60367 entries, 0 to 60366
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   overall      60367 non-null  int64  
 1   vote         60367 non-null  int64  
 2   verified     60367 non-null  int64  
 3   asin         60367 non-null  object 
 4   reviewText   60367 non-null  object 
 5   summary      60362 non-null  object 
 6   category     60367 non-null  object 
 7   description  60367 non-null  object 
 8   title        60367 non-null  object 
 9   rank         60367 non-null  object 
 10  price        60367 non-null  float64
 11  rank_length  60367 non-null  int64  
 12  rank_1       60367 non-null  int64  
 13  rank_2       60367 non-null  int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 6.9+ MB


In [5]:
df_laptops.head()

,overall,vote,verified,asin,reviewText,summary,category,description,title,rank,price,rank_length,rank_1,rank_2
0,5,3,0,B00005NBIS,you can't have everything at least not at 1199...,The perfect compromise,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
1,5,3,0,B00005NBIS,you can't have everything at least not at 1199...,The perfect compromise,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
2,5,2,0,B00005NBIS,i bought the one with a 20 gb hard drive and i...,Basic 500 MHz iBook,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
3,5,2,0,B00005NBIS,i bought the one with a 20 gb hard drive and i...,Basic 500 MHz iBook,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593
4,5,34,0,B00005NBIS,apple has reinvented the ibook as one of the b...,A great laptop! *****,"[Electronics, Computers &amp; Accessories, Com...",['apple ibook g3-500 10gb 64mb cd 121-svga ene...,"Apple iBook Laptop (500-MHz PowerPC G3, 64 MB ...","[>#59,248 in Computers &amp; Accessories (See ...",0.0,2,59248,15593


In [6]:
df_laptops['rank_1'].value_counts()

0         2742
4464      1302
4198       938
3664       598
5731       584
          ... 
47659        5
32224        5
152791       4
58299        4
238718       4
Name: rank_1, Length: 2402, dtype: int64

In [7]:
df_laptops['verified'].value_counts()

1    47430
0    12937
Name: verified, dtype: int64

In [8]:
df_laptops = df_laptops.fillna(0)
df_laptops['vote'] = df_laptops.vote.replace(',','', regex=True)

In [9]:
len(pd.unique(df_laptops['asin']))

2584

# Loading LDA Models

In [10]:
import gensim
from gensim import corpora, models

In [11]:
lda_model_laptop_review = gensim.models.ldamodel.LdaModel.load(base_dir+'lda_model_laptop_review_15.model')  


In [12]:
from pprint import pprint
 
# Print the Keyword in the 20 topics
pprint(lda_model_laptop_review.print_topics())

[(0,
  '0.052*"key" + 0.051*"keyboard" + 0.037*"use" + 0.025*"touch" + 0.022*"pad" '
  '+ 0.021*"button" + 0.021*"mouse" + 0.019*"right" + 0.017*"screen" + '
  '0.014*"click"'),
 (1,
  '0.077*"usb" + 0.073*"port" + 0.038*"device" + 0.037*"wifi" + 0.030*"power" '
  '+ 0.029*"connect" + 0.027*"wireless" + 0.025*"connection" + '
  '0.022*"bluetooth" + 0.018*"hdmi"'),
 (2,
  '0.103*"tablet" + 0.096*"great" + 0.095*"good" + 0.055*"work" + '
  '0.051*"price" + 0.050*"product" + 0.047*"excellent" + 0.019*"far" + '
  '0.018*"value" + 0.018*"fast"'),
 (3,
  '0.054*"buy" + 0.037*"love" + 0.031*"use" + 0.030*"year" + 0.022*"get" + '
  '0.019*"great" + 0.018*"old" + 0.018*"good" + 0.017*"purchase" + '
  '0.015*"new"'),
 (4,
  '0.029*"game" + 0.019*"high" + 0.018*"play" + 0.016*"graphic" + '
  '0.016*"performance" + 0.015*"run" + 0.014*"well" + 0.013*"price" + '
  '0.013*"processor" + 0.013*"gaming"'),
 (5,
  '0.063*"screen" + 0.035*"quality" + 0.029*"feel" + 0.024*"build" + '
  '0.020*"keyboard" +

In [13]:
#lda_model_laptop_des = gensim.models.ldamodel.LdaModel.load(base_dir+'lda_model_laptop_des.model')  

In [14]:
#from pprint import pprint
 
# Print the Keyword in the 20 topics
#pprint(lda_model_laptop_des.print_topics())

In [15]:
with open(base_dir+'laptop_review_corpus.pkl','rb') as f:
  corpus=pickle.load(f)

In [16]:
#with open(base_dir+'laptop_des_corpus.pkl','rb') as f:
#  corpus_des=pickle.load(f)

# Creating Topic Vectors

In [21]:
review_topics = []
des_topics =[]
vote = []
topic_vec=[]
util_rev = []
hedo_rev = []
amb_rev = []
util_des = []
hedo_des = []
amb_des = []
for i in range(len(df_laptops)):
    top_topics_review = lda_model_laptop_review.get_document_topics(corpus[i], minimum_probability=0.0)
    #top_topics_des = lda_model_laptop_des.get_document_topics(corpus_des[i], minimum_probability=0.0)
    topic_vec_review = [top_topics_review[i][1] for i in range(15)]
    topic_vec_review_sum = sum(topic_vec_review)
    #topic_vec_des = [top_topics_des[i][1] for i in range(8)]
    #topic_vec_des_sum = sum(topic_vec_des)
    util_rev.append((topic_vec_review[1]+topic_vec_review[6]+topic_vec_review[9]+topic_vec_review[11]+topic_vec_review[14])/topic_vec_review_sum)
    hedo_rev.append((topic_vec_review[3]+topic_vec_review[4]+topic_vec_review[5]+topic_vec_review[10]+topic_vec_review[12])/topic_vec_review_sum)
    amb_rev.append((topic_vec_review[0]+topic_vec_review[2]+topic_vec_review[7]+topic_vec_review[8]+topic_vec_review[13])/topic_vec_review_sum)
    #util_des.append((topic_vec_des[1]+topic_vec_des[3]+topic_vec_des[6]+topic_vec_des[7])/topic_vec_des_sum)
    #hedo_des.append((topic_vec_des[0]+topic_vec_des[4])/topic_vec_des_sum)
    #amb_des.append((topic_vec_des[2]+topic_vec_des[5])/topic_vec_des_sum)
    #topic_vec.extend([df_inner.iloc[i].helpful])
    review_topics.append(topic_vec_review)
    #des_topics.append(topic_vec_des)

In [22]:
#topic_vec_des

In [23]:
topic_vec_review

[0.0026666685,
 0.082666665,
 0.0026666685,
 0.0026666704,
 0.11422989,
 0.002666671,
 0.0026666673,
 0.0026666669,
 0.0026666706,
 0.0026666715,
 0.77110344,
 0.0026666692,
 0.0026666855,
 0.0026666669,
 0.0026666685]

In [30]:
max_util = pd.Series(util_rev).idxmax()

In [31]:
max_hedo= pd.Series(hedo_rev).idxmax()

In [32]:
max_amb = pd.Series(amb_rev).idxmax()

In [36]:
df_laptops['reviewText'][max_util]

"i bought my ideapad a couple of months ago have been trying to get a vga adapter for it it has either a mini vga or mini display port on the left side i called lenovo 3 weeks ago and ordered an adapter cable when it didn't arrive i called them and they had no record of my order so i asked if i could order the part again they were not familiar with the connector on their computer but sold me an hdmi to vga adapter after giving my credit card number i asked when to expect delivery they said it would ship in 5-8 days i said how long to receive they said shipping is free and they can't say i offered to pay extra for shipping since i have a presentation where i need to plug into a projector with vga ended up cancelling my order"

In [37]:
df_laptops['reviewText'][max_hedo]

'this laptop feels so premium in my opinion i prefer this over the macbook air because it feels premium and it looks premium this screen is really good for 200$ less than the macbook air it is so worth the money because it performs better than the macbook air it feels like premium metal externally this laptop for being 133 inch is not that small its the perfect size when it comes to portablility its also blazing fast for a intel core i5 8mb ram processor you will have a better experience with this laptop over the macbook air or any ultrabook in my opinion'

In [38]:
df_laptops['reviewText'][max_amb]

'works perfect for me  yes the mouse pad is annoying but i attached a wireless mouse for my own convenience  oh and the headphone jack is in the back which seems like a stupid design idea  but works fine and i am happy with my purchase'

# OLS Regression Model

In [ ]:
#import numpy as np
#import statsmodels.api as sm # import statsmodels 
#import pandas as pd

#df_laptops['review_length'] = df_laptops.reviewText.str.len()

In [ ]:
diff_hedo = np.subtract(hedo_des, hedo_rev)
diff_util = np.subtract(util_des, util_rev)
diff_amb = np.subtract(amb_des, amb_rev)

In [39]:
import numpy as np
import statsmodels.api as sm # import statsmodels 

#x1 = np.array(hedo_des)
#x2= np.array(util_des)
#x3 = np.array(amb_des)
x1 = np.array(hedo_rev)
x2= np.array(util_rev)
x3 = np.array(amb_rev)
#x1 = np.array(diff_hedo)
#x2 = np.array(diff_util)
#x3 = np.array(diff_amb)
#x1 = np.array(review_topics)
x4= np.array(df_laptops[['overall']])
x5= np.array(df_laptops[['verified']])
#x6= np.array(df_laptops[['rank_1']])
x7= np.array(df_laptops[['rank_2']])
X = np.column_stack((x1, x2, x3, x4, x5, x7))
y = np.array((df_laptops.vote.astype(int)))

# Note the difference in argument order
model = sm.OLS(y, X).fit() ## sm.OLS(output, input)
#model2 = sm.OLS(y1, X).fit() ## sm.OLS(output, input)
predictions = model.predict(X)
# Print out the statistics
model.summary()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     67.86
Date:                Fri, 27 Nov 2020   Prob (F-statistic):           5.61e-71
Time:                        20:42:14   Log-Likelihood:            -3.1395e+05
No. Observations:               60367   AIC:                         6.279e+05
Df Residuals:                   60361   BIC:                         6.280e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.1402      0.937      7.623      0.000       5.304       8.976
x2            10.9064      0.773     14.101      0.000       9.390      12.422
x3             7.6604      1.031      7.430      0.000       5.640       9.681
x4             0.8833      0.152      5.794      0.000       0.584       1.182
x5            -7.8362      0.442    -17.724      0.000      -8.703      -6.970
x6         -7.125e-05    2.2e-05     -3.236      0.001      -0.000   -2.81e-05
==============================================================================
Omnibus:                   218526.111   Durbin-Watson:                   1.899
Prob(Omnibus):                  0.000   Jarque-Bera (JB):     246966677551.646
Skew:                          74.836   Prob(JB):                         0.00
Kurtosis:                    9910.759   Cond. No.                     7.32e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""